# <img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Monitor your ML Models using Watson OpenScale and WML on Cloud Pak for Data

## 1. Setup the Notebook Environment

## 1.1 Install the necessary packages

### Watson OpenScale Python SDK

In [ ]:
!pip install ibm-ai-openscale

### Scikit-learn version 0.22


In [ ]:
!pip install scikit-learn==0.22.0

### Watson Machine Learning Python SDK

In [ ]:
!pip install --upgrade watson-machine-learning-client-V4==1.0.93 | tail -n 1

Restart the Notebook after Installing the required packages. By clicking on `Kernel>Restart`

## 1.2 Import Packages

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn import preprocessing
from sklearn import svm, metrics
from scipy import sparse
from watson_machine_learning_client import WatsonMachineLearningAPIClient
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import json
import ibm_db


import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

## 2. Configuration

### 2.1 Global Variables

In [ ]:
MODEL_NAME="telco_churn_monitored_srs"
DEPLOYMENT_NAME="telco_churn_monitored_srs_deployment"
# Ensure you create a an empty Schema and store the name in this variable
# SCHEMA_NAME="JLQ22844"

# Enter the Deployment Space you have associated project with 
dep_name="telco_churn_deployment_space_srs"

### 2.2 Add Dataset

Select the `Insert Pandas Dataframe` option, after selecting the below cell. Ensure the variable name is `df_data_1`

In [ ]:
# Place cursor below and insert the Pandas DataFrame for the Telco churn data


### 2.3 Update your AIOS Credentials

In [ ]:
WOS_CREDENTIALS={
    "url" : os.environ['RUNTIME_ENV_APSX_URL'],
    "username":"XXXXXXX",
    "password":"XXXXXXX"
}

### 2.4 Input your WML Credentials 


In [ ]:
import sys,os,os.path


# WML_CREDENTIALS = {
# "token": os.environ['USER_ACCESS_TOKEN'],
# "instance_id" : "wml_local",
# "url" : os.environ['RUNTIME_ENV_APSX_URL'],
# "version": "3.0.0"
# }
WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='3.0.0'

### 2.5 Add your Db credentials

#### These Db credentials are needed ONLY if you have NOT configured your `OpenScale Datamart`.

In [ ]:

# DATABASE_CREDENTIALS = {
#     "hostname": "dashdb-txn-sbox-yp-dal09-11.services.dal.bluemix.net",
#     "username": "tzm22305",
#     "password": "s2knhr3znx-c5s03",
#     "port": 50000,
#     "db": "BLUDB",
    
# }


## 3. Create the Customer Churn Model using Scikit-Learn

In [ ]:
X=df_data_1.drop(['Churn'], axis=1)
y=df_data_1.loc[:, 'Churn']

In [ ]:
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)


In [ ]:
'''Add a categorical transformer to your model pipeline. 
    You will need to add a label encoder into the model pipeline before storing it into WML '''

categorical_features = ["gender", "SeniorCitizen", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity",
                        "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod"]
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)])

In [ ]:
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', svm.SVC(kernel='linear'))])

In [ ]:
model = pipeline.fit(X_train,y_train)

In [ ]:
from sklearn.metrics  import accuracy_score, classification_report
y_scores = model.predict(X_test)
print(accuracy_score(y_test, y_scores))
print(classification_report(y_test, y_scores))

## 4. Create a new Deployment

In [ ]:
client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

In [ ]:
meta_props={
 client.repository.ModelMetaNames.NAME: MODEL_NAME,
 client.repository.ModelMetaNames.RUNTIME_UID: "scikit-learn_0.22-py3.6",
 client.repository.ModelMetaNames.TYPE: "scikit-learn_0.22",
}

In [ ]:

project_id = os.environ['PROJECT_ID']
client.set.default_project(project_id)

In [ ]:
def guid_from_space_name(client, space_name):

    instance_details = client.service_instance.get_details()

    space = client.spaces.get_details()
    res=[]
    for item in space['resources']: 
        if item['entity']["name"] == space_name:
            res=item['metadata']['guid']

    return res

In [ ]:
# Enter the name of your deployment space of the current project

space_uid = guid_from_space_name(client, dep_name)

In [ ]:
space_uid

In [ ]:
client.set.default_space(space_uid)


### Store, Deploy and Score your Custom WML Model

In [ ]:
deploy_meta = {
     client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
     client.deployments.ConfigurationMetaNames.ONLINE: {}
 }

In [ ]:
## Store the model on WML
published_model = client.repository.store_model(pipeline,
                                             meta_props=meta_props,
                                             training_data=X_train,
                                             training_target=y_train
                                                )


In [ ]:
published_model_uid = client.repository.get_model_uid(published_model)

In [ ]:
## Create a Deployment for your stored model

created_deployment = client.deployments.create(published_model_uid, meta_props=deploy_meta)

In [ ]:

scoring_endpoint = None
deployment_uid=created_deployment['metadata']['guid']

## 5. Setup your Watson Openscale Dashboard 

### 5.1 Create the Watson Openscale Client

In [ ]:
ai_client = APIClient4ICP(aios_credentials=WOS_CREDENTIALS)
ai_client.version

### 5.2 Setup the Datamart on AI OpenScale

In [ ]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    print('Using existing external datamart')
except:
    print('Setting up external datamart')
    ai_client.data_mart.setup(db_credentials=DATABASE_CREDENTIALS, schema=SCHEMA_NAME)

In [ ]:
data_mart_details = ai_client.data_mart.get_details()

In [ ]:
data_mart_details

### 5.3 Add your Machine Learning Provider

If you have already bound the ML Provider to the Openscale instance, then just retrieve the binding_uid, by commenting first line and uncommenting the second line

In [ ]:
WML_CREDENTIALS

In [ ]:
binding_uid = ai_client.data_mart.bindings.add('WML instance - churn_srs', WatsonMachineLearningInstance4ICP(wml_credentials=WML_CREDENTIALS))


In [ ]:
ai_client.data_mart.bindings.list_assets()

### 5.4 Perform Initial Scoring for your Model Deployment


In [ ]:
score=X_test.tail(20)
score

In [ ]:
scoring_data=list(list(x) for x in zip(*(score[x].values.tolist() for x in score.columns)))
scoring_data

In [ ]:
fields=list(X_test.columns)
print(len(fields))
fields, scoring_data[0]

In [ ]:

job_payload = {
client.deployments.ScoringMetaNames.INPUT_DATA: [{
 'values': scoring_data
}]
}
print(job_payload)

In [ ]:
scoring_response = client.deployments.score(deployment_uid, job_payload)

print(scoring_response)

### 5.5 Create a new Subscription 

In [ ]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    published_model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Churn',
    prediction_column='prediction',
    probability_column='prediction_probability',
    categorical_columns=["gender", "SeniorCitizen", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity",
                        "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod"],
    feature_columns = ["gender", "SeniorCitizen", "Partner", "Dependents", "PhoneService", "MultipleLines", "InternetService", "OnlineSecurity",
                        "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "tenure","TotalCharges","MonthlyCharges"],
))

In [ ]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

### 5.6 Perform Inital Payload Logging
Note: You may re-use this code snippet by modifying the request_data variable to perform payload logging after finishing the initial dashboard setup

In [ ]:
fields=list(X_test.columns)

request_data = {
    "fields": fields,
    "values": scoring_data
  }
request_data

**<font color='red'><< REPLACE subscription_uid BELOW with the uid for your subscription. For e.g.<br/>subscription_uid="644e4e6d-8a82-4f07-9489-381d44469a23" >></font>**

In [ ]:
## From the output of the above table choose your model name and copy the uid against it. Store the uid in the subscription_uid variable


subscription_uid="5664ad39-2363-44bd-bbdc-fbce71bbb575"
from ibm_ai_openscale import APIClient4ICP
from ibm_ai_openscale.supporting_classes import PayloadRecord


subscription = ai_client.data_mart.subscriptions.get(subscription_uid=subscription_uid)


records = [PayloadRecord(request=request_data, response=scoring_response, response_time=18), 
                PayloadRecord(request=request_data, response=scoring_response, response_time=12)]

subscription.payload_logging.store(records=records)

### 5.7 Setup Quality Monitoring

```NOTE: If you are using the dataset provided in the dashboard, leave the threshold monitors to these values. However, if you are using your own dataset, you can play around with the threshold value (value b/w 0 and 1) according to your requirement.```

In [ ]:
time.sleep(5)
subscription.quality_monitoring.enable(threshold=0.90, min_records=5)

### 5.8 Log Feedback Data to your Subscription

In [ ]:
feedback_data_raw=pd.concat([X_test,y_test],axis=1)
feedback_data_raw

In [ ]:
feedback_data=feedback_data_raw.tail(20).values.tolist()
feedback_data

In [ ]:
feedback_scoring={
    "data":feedback_data
}

In [ ]:
subscription.feedback_logging.store(feedback_scoring['data'])


In [ ]:
subscription.feedback_logging.show_table()

#### Run an inital quality test

In [ ]:
run_details = subscription.quality_monitoring.run(background_mode=False)

In [ ]:
subscription.quality_monitoring.show_table()

In [ ]:
%matplotlib inline

quality_pd = subscription.quality_monitoring.get_table_content(format='pandas')
quality_pd.plot.barh(x='id', y='value');

### 5.9 Setup the Fairness Monitors

The code below configures fairness monitoring for our model. It turns on monitoring for two features, _conds(Weather Condition) and Traffic for the cell tower. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 5 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [ ]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("gender", majority=['Male'], minority=['Female'], threshold=0.95),
            ],
            favourable_classes=["No"],
            unfavourable_classes=["Yes"],
            min_records=50,
            training_data=df_data_1
        )

In [ ]:
score2=X_test.head(50)

scoring_data2=list(list(x) for x in zip(*(score2[x].values.tolist() for x in score2.columns)))

fields2=list(X_test.columns)

job_payload2 = {
client.deployments.ScoringMetaNames.INPUT_DATA: [{
 'values': scoring_data2
}]
}

scoring_response2 = client.deployments.score(deployment_uid, job_payload2)


request_data2 = {
    "fields": fields,
    "values": scoring_data2
  }

records2 = [PayloadRecord(request=request_data2, response=scoring_response2, response_time=18), 
                PayloadRecord(request=request_data2, response=scoring_response2, response_time=12)]

subscription.payload_logging.store(records=records2)

In [ ]:
time.sleep(20)

run_details = subscription.fairness_monitoring.run(background_mode=False)

In [ ]:
time.sleep(5)

subscription.fairness_monitoring.show_table()

## 6.0 Custom monitors and metrics


### 6.1 Register custom monitor

In [ ]:
def get_definition(monitor_name):
    monitors_definitions = ai_client.data_mart.monitors.get_details()['monitor_definitions']
    
    for definition in monitors_definitions:
        if monitor_name == definition['entity']['name']:
            return definition
    
    return None

Change `monitor_name` to something unique.

In [ ]:
from ibm_ai_openscale.supporting_classes import Metric, Tag

monitor_name = 'custom_monitor_shivam'
metrics = [Metric(name='sensitivity', lower_limit_default=0.8), Metric(name='specificity', lower_limit_default=0.75)]
tags = [Tag(name='region', description='customer geographical region')]

existing_definition = get_definition(monitor_name)

if existing_definition is None:
    my_monitor = ai_client.data_mart.monitors.add(name=monitor_name, metrics=metrics, tags=tags)
else:
    my_monitor = existing_definition

### 6.1.1 Get monitors uids and details

In [ ]:
monitor_uid = my_monitor['metadata']['guid']

print(monitor_uid)

In [ ]:
my_monitor = ai_client.data_mart.monitors.get_details(monitor_uid=monitor_uid)
print('monitor definition details', my_monitor)

## 6.2 Enable custom monitor for subscription

In [ ]:
from ibm_ai_openscale.supporting_classes import Threshold

thresholds = [Threshold(metric_uid='sensitivity', lower_limit=0.9)]
subscription.monitoring.enable(monitor_uid=monitor_uid, thresholds=thresholds)

### 6.2.1 Get monitor configuration details

In [ ]:
subscription.monitoring.get_details(monitor_uid=monitor_uid)

## 6.3 Storing custom metrics

In [ ]:
metrics = {"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}

subscription.monitoring.store_metrics(monitor_uid=monitor_uid, metrics=metrics)

### 6.3.1 List and get custom metrics

In [ ]:
subscription.monitoring.show_table(monitor_uid=monitor_uid)

In [ ]:
custom_metrics = subscription.monitoring.get_metrics(monitor_uid=monitor_uid, deployment_uid='credit')
custom_metrics

In [ ]:
custom_metrics_pandas = subscription.monitoring.get_table_content(monitor_uid=monitor_uid)

%matplotlib inline
custom_metrics_pandas.plot.barh(x='id', y='value');

## 7.0 Payload analytics


### 7.1 Run data distributions calculation

In [ ]:
from datetime import datetime

start_date = "2018-01-01T00:00:00.00Z"
end_date = datetime.utcnow().isoformat() + "Z"

sex_distribution = subscription.payload_logging.data_distribution.run(
            start_date=start_date,
            end_date=end_date,
            group=['prediction', 'gender'],
            agg=['count'])

### 7.2 Get data distributions as pandas dataframe


In [ ]:
sex_distribution_run_uid = sex_distribution['id']
distributions_pd = subscription.payload_logging.data_distribution.get_run_result(run_id=sex_distribution_run_uid, format='pandas')
distributions_pd

In [ ]:
subscription.payload_logging.data_distribution.show_chart(sex_distribution_run_uid);

## 8. Identify transactions for Explainability

In [ ]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(training_data=df_data_1)

In [ ]:
payload_data = subscription.payload_logging.get_table_content(limit=60)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability'])

### Add some more Payload (Optional for populating your dashboard)

If you wish to add some Payload Data. Take different sections of your test dataset and send to OpenScale as shown below-

In [ ]:
score100=X_test.head(100)
score100

In [ ]:
scoring_data100=list(list(x) for x in zip(*(score100[x].values.tolist() for x in score100.columns)))
scoring_data100

In [ ]:
fields=list(X_test.columns)
print(len(fields))
fields, scoring_data100[0]

In [ ]:
request_data100 = {
    "fields": fields,
    "values": scoring_data100
  }
request_data100

In [ ]:
records100 = [PayloadRecord(request=request_data100, response=scoring_response, response_time=18), 
                PayloadRecord(request=request_data100, response=scoring_response, response_time=12)]

subscription.payload_logging.store(records=records100)